El objetivo de este código es implementar el algoritmo MLP, separando los datos por sujeto.
Va a buscar los datos en una carpeta que se encuentre en el mismo directorio que se llame *datos_baseline*.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [2]:
# instalar comet ml en caso que sea necesario
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.8.0
    Uninstalling websocket-client-1.8.0:
      Successfully uninstalled websocket-client-1.8.0
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [3]:
# Importo las librerías
from lightgbm import LGBMClassifier
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import time
from comet_ml import Experiment
import joblib
import matplotlib.pyplot as plt
import sys
sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [5]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto los datos
features = load("FEATURES_W200_I50_5050.joblib")
label = load("label_W200_I50_5050.joblib")

features_array = np.array(features)
label_array = np.array(label)

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [6]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto particiones
folds_ind_lista = []
sujetos_array_lista = []

for i in range(0,10):
  folds_ind_lista.append(joblib.load('folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl'))
  sujetos_array_lista.append(joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl'))

# Aclaracion: sujetos_array va de 0 a 9. Por lo que el sujeto 0 en realidad es el 1 y asi sucesivamente

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [7]:
# agrego una función para transformar a 1 todas las clases distintas de 0, es decir, todas las que no sean gesto
def go_to_binary_problem(label):
  label_copy = label.copy()

  ind_gestos = np.where(label_copy[:, 1]!=0)[0]
  label_copy[ind_gestos, 1] = 1
  print('Proporcion clase 1: '+str(len(ind_gestos)/len(label_copy)))
  return label_copy

label_array = go_to_binary_problem(label_array)

Proporcion clase 1: 0.5


In [8]:
for j in range(0,10):

  # Particion de datos de esta iteracion
  folds_ind = folds_ind_lista[j]
  sujetos_array = sujetos_array_lista[j]

  # Voy a usar como sujeto (o fold, es lo mismo) de test el que aparece primero. Mirar primer elemento de sujetos_array, y tener presente
  # la aclaracion hecha en el bloque de codigo anterior

  # por el criterio elegido, se que el primer fold (o sujeto) es para test, asi que lo saco
  folds_ind_test = folds_ind[0]

  # Uno los datos de los folds de entrenamiento
  # me quedo con los folds (o sujetos) de train quitando el primero (posicion 0)
  folds_ind_train = folds_ind[1:]
  # concateno los arrays (folds) en uno solo
  folds_ind_train_array = np.concatenate(folds_ind_train)
  # aplano el array resultante
  folds_ind_train_array = folds_ind_train_array.ravel()

  # Si tuviera mas de un fold de test tendria que hacer lo mismo que hice recien para train, pero no lo hago porque solo tengo uno

  # Defino X_train, y_train, X_test, y_test
  X_train = features_array[folds_ind_train_array, :]
  y_train = label_array[folds_ind_train_array, 1]
  X_test = features_array[folds_ind_test, :]
  y_test = label_array[folds_ind_test, 1]

  start_time = time.time()    # comienzo a medir el tiempo
  # Parametros de entrada de la red neuronal que tenemos que definir
  # Definimos a mano la cantidad de features
  featureNum = 15
  chNum = 16
  # Numero de columnas que va a haber en features
  inputDim = chNum*featureNum
  # Numero de gestos contando el reposo
  clasNum = np.max(label_array[:, 1]) + 1

  # Architecture 250 - 500 - 250
  clf = cl.Clasificador(model= "mlp", arch = [250,500,250], catNum = clasNum, featureNum = inputDim)
  clf.train(X_train, y_train)

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo

  nombre = 'baseline_mlp_sep_sub' + str(j+1) + '_test_50_50'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'mlp', 'sep_sub', 'choose_test', '50_50']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("partition_array", sujetos_array)   # Guarda el arreglo en el experimento
  exp.log_text("El primer sujeto de partition_array es el de test, el resto son todos de train. Se usó la particion que se guarda en datos_baseline")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

Epoch 1/50
3686/3686 [==============================] - 28s 6ms/step - loss: 0.3435 - accuracy: 0.8564
Epoch 2/50
3686/3686 [==============================] - 19s 5ms/step - loss: 0.3230 - accuracy: 0.8656
Epoch 3/50
3686/3686 [==============================] - 21s 6ms/step - loss: 0.3129 - accuracy: 0.8698
Epoch 4/50
3686/3686 [==============================] - 19s 5ms/step - loss: 0.3059 - accuracy: 0.8736
Epoch 5/50
3686/3686 [==============================] - 22s 6ms/step - loss: 0.3002 - accuracy: 0.8759
Epoch 6/50
3686/3686 [==============================] - 21s 6ms/step - loss: 0.2954 - accuracy: 0.8789
Epoch 7/50
3686/3686 [==============================] - 18s 5ms/step - loss: 0.2911 - accuracy: 0.8803
Epoch 8/50
3686/3686 [==============================] - 22s 6ms/step - loss: 0.2875 - accuracy: 0.8813
Epoch 9/50
3686/3686 [==============================] - 21s 6ms/step - loss: 0.2837 - accuracy: 0.8838
Epoch 10/50
3686/3686 [==============================] - 19s 5ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/2ca2777a20c9431e954deb1d40fa0389

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub1_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/2ca2777a20c9431e954deb1d40fa0389
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.8684778697

Epoch 1/50
3623/3623 [==============================] - 21s 6ms/step - loss: 0.3476 - accuracy: 0.8547
Epoch 2/50
3623/3623 [==============================] - 20s 6ms/step - loss: 0.3286 - accuracy: 0.8624
Epoch 3/50
3623/3623 [==============================] - 20s 5ms/step - loss: 0.3168 - accuracy: 0.8679
Epoch 4/50
3623/3623 [==============================] - 19s 5ms/step - loss: 0.3089 - accuracy: 0.8719
Epoch 5/50
3623/3623 [==============================] - 24s 7ms/step - loss: 0.3027 - accuracy: 0.8738
Epoch 6/50
3623/3623 [==============================] - 19s 5ms/step - loss: 0.2970 - accuracy: 0.8759
Epoch 7/50
3623/3623 [==============================] - 19s 5ms/step - loss: 0.2934 - accuracy: 0.8766
Epoch 8/50
3623/3623 [==============================] - 20s 5ms/step - loss: 0.2897 - accuracy: 0.8799
Epoch 9/50
3623/3623 [==============================] - 20s 5ms/step - loss: 0.2855 - accuracy: 0.8807
Epoch 10/50
3623/3623 [==============================] - 18s 5ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/ed19f9beadf14496862e849ea2c16e6b

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub2_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/ed19f9beadf14496862e849ea2c16e6b
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.8924272436

Epoch 1/50
3686/3686 [==============================] - 22s 6ms/step - loss: 0.3541 - accuracy: 0.8517
Epoch 2/50
3686/3686 [==============================] - 19s 5ms/step - loss: 0.3345 - accuracy: 0.8608
Epoch 3/50
3686/3686 [==============================] - 19s 5ms/step - loss: 0.3228 - accuracy: 0.8659
Epoch 4/50
3686/3686 [==============================] - 19s 5ms/step - loss: 0.3112 - accuracy: 0.8722
Epoch 5/50
3686/3686 [==============================] - 22s 6ms/step - loss: 0.3045 - accuracy: 0.8760
Epoch 6/50
3686/3686 [==============================] - 21s 6ms/step - loss: 0.2985 - accuracy: 0.8783
Epoch 7/50
3686/3686 [==============================] - 18s 5ms/step - loss: 0.2948 - accuracy: 0.8803
Epoch 8/50
3686/3686 [==============================] - 19s 5ms/step - loss: 0.2907 - accuracy: 0.8813
Epoch 9/50
3686/3686 [==============================] - 21s 6ms/step - loss: 0.2874 - accuracy: 0.8825
Epoch 10/50
3686/3686 [==============================] - 19s 5ms/step - l

...

Epoch 1/50
3626/3626 [==============================] - 20s 5ms/step - loss: 0.3290 - accuracy: 0.8655
Epoch 2/50
3626/3626 [==============================] - 21s 6ms/step - loss: 0.3075 - accuracy: 0.8751
Epoch 3/50
3626/3626 [==============================] - 21s 6ms/step - loss: 0.2938 - accuracy: 0.8811
Epoch 4/50
3626/3626 [==============================] - 19s 5ms/step - loss: 0.2856 - accuracy: 0.8845
Epoch 5/50
3626/3626 [==============================] - 20s 5ms/step - loss: 0.2795 - accuracy: 0.8874
Epoch 6/50
3626/3626 [==============================] - 21s 6ms/step - loss: 0.2736 - accuracy: 0.8896
Epoch 7/50
3626/3626 [==============================] - 20s 5ms/step - loss: 0.2683 - accuracy: 0.8917
Epoch 8/50
3626/3626 [==============================] - 20s 5ms/step - loss: 0.2636 - accuracy: 0.8931
Epoch 9/50
3626/3626 [==============================] - 22s 6ms/step - loss: 0.2587 - accuracy: 0.8949
Epoch 10/50
3626/3626 [==============================] - 20s 6ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/b1d5a24644014f6cb5b4d6ce1dedd0a4

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub9_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/b1d5a24644014f6cb5b4d6ce1dedd0a4
COME

Epoch 1/50
3616/3616 [==============================] - 21s 6ms/step - loss: 0.3386 - accuracy: 0.8584
Epoch 2/50
3616/3616 [==============================] - 19s 5ms/step - loss: 0.3178 - accuracy: 0.8679
Epoch 3/50
3616/3616 [==============================] - 20s 6ms/step - loss: 0.3062 - accuracy: 0.8732
Epoch 4/50
3616/3616 [==============================] - 21s 6ms/step - loss: 0.2948 - accuracy: 0.8784
Epoch 5/50
3616/3616 [==============================] - 20s 5ms/step - loss: 0.2895 - accuracy: 0.8803
Epoch 6/50
3616/3616 [==============================] - 20s 5ms/step - loss: 0.2834 - accuracy: 0.8833
Epoch 7/50
3616/3616 [==============================] - 22s 6ms/step - loss: 0.2788 - accuracy: 0.8852
Epoch 8/50
3616/3616 [==============================] - 20s 5ms/step - loss: 0.2743 - accuracy: 0.8873
Epoch 9/50
3616/3616 [==============================] - 20s 5ms/step - loss: 0.2724 - accuracy: 0.8881
Epoch 10/50
3616/3616 [==============================] - 22s 6ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/c08be4062231433e8d0c797181a9b79d

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub10_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/c08be4062231433e8d0c797181a9b79d
COM

<Figure size 640x480 with 0 Axes>

In [ ]:
start_time = time.time()    # comienzo a medir el tiempo

# Crear una instancia de LGBMClassifier
clf = LGBMClassifier()

# Entrenar el clasificador
clf.fit(X_train, y_train, eval_metric='logloss')

# Finalizo la medida del tiempo y calculo el tiempo de entrenamiento
end_time = time.time()
training_time = end_time - start_time

# # Realizar predicciones en el conjunto de validación
# y_pred = clf.predict(X_val)

# # Calcular la precisión
# accuracy = accuracy_score(y_val, y_pred)
# print(f'Accuracy: {accuracy}')

# Obtener las iteraciones del mejor modelo
best_iteration = clf.best_iteration_

# Imprimir la advertencia si existe
if hasattr(clf, 'best_iteration_') and clf.best_iteration_ is None:
    print("[LightGBM] [Warning] No further splits with positive gain, best gain: -inf")



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.216863 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50308
[LightGBM] [Info] Number of data points in the train set: 37655, number of used features: 240
[LightGBM] [Info] Start training from score -2.565481
[LightGBM] [Info] Start training from score -2.592793
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.591020
[LightGBM] [Info] Start training from score -2.573805
[LightGBM] [Info] Start training from score -2.543614
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.376846
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.740986
Accurac

Voy a guardar el clasificador, y evaluar distintas métricas: accuracy, precision y recall. Voy a hacer una matriz de confusión.
Por otra parte, voy a guardar la partición de los datos para hacer reproducible el experimento.

In [ ]:
# Guardar el modelo entrenado en un archivo
joblib.dump(bst, 'baseline_gbm_sep_sub_r1.pkl')

# Predecir en el conjunto de test
y_pred = bst.predict(X_test)

# Calcular métricas de desempeño
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar las métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Guardar la imagen de la matriz de confusión
plt.savefig("confusion_matrix.png")

Voy a guardar las métricas calculadas en un experimento en Comet

In [ ]:
# Conectar con Comet
API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO' # Se puede encontrar en Settings(Arriba a la derecha en Comet)

# Crear un experimento con mi API KEY
exp = Experiment(api_key=API_KEY,
                 project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                 auto_param_logging=False)
exp.set_name('baseline_gbm_sep_sub_r1') # Nombre de este experimento
exp.add_tags(['baseline', 'gbm', 'sep_sub']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(y_test, y_pred)
exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
exp.log_text("Primeros dos sujetos --> test, tercero y cuarto --> validación, resto --> train. \n Corresponde a la primera ronda que entreno con estos parámetros. ")   # Comentario del experimento

In [ ]:
# Subir el modelo
exp.log_model(name="baseline_gbm_sep_sub_r1", file_or_folder="baseline_gbm_sep_sub_r1.pkl")
exp.end()

Junto todo en un bloque de código

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"

for k in range(1,2):
  label = np.array(label)
  cantSujetos = np.max(label[:, 2])
  sujeto_test = k
  sorteo = np.random.permutation(cantSujetos) + 1
  sorteo_sin_test = np.delete(sorteo, np.where(sorteo == sujeto_test))
  indices_test = list(np.where(label[:, 2]==sujeto_test)[0])
  indices_val = list(np.where(label[:, 2]==sorteo_sin_test[0])[0])
  indices_val.extend(list(np.where(label[:, 2]==sorteo_sin_test[1])[0]))

  indices_train = []
  for j in sorteo_sin_test[2:]:
      indices_train.extend(np.where(label[:, 2]==j)[0])
  features = np.array(features)

  X_train = features[indices_train, :]
  y_train = label[indices_train, 1]
  X_val = features[indices_val, :]
  y_val = label[indices_val, 1]
  X_test =  features[indices_test, :]
  y_test = label[indices_test, 1]

  start_time = time.time()

  # implemento gbm
  clf = LGBMClassifier()
  clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss')

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo
  bst = clf.best_iteration_

  nombre = 'baseline_gbm_sep_sub' + str(k) + '_testing'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'gbm', 'sep_sub', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
  exp.log_text("Se fija el sujeto de test y se sortea el resto. Los primeros dos sujetos del sorteo son los de validación.")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

/content/drive/My Drive/repo_tesis/archivos_generados_codigos
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.541994 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50345
[LightGBM] [Info] Number of data points in the train set: 45226, number of used features: 240
[LightGBM] [Info] Start training from score -2.565215
[LightGBM] [Info] Start training from score -2.600029
[LightGBM] [Info] Start training from score -2.605103
[LightGBM] [Info] Start training from score -2.578238
[LightGBM] [Info] Start training from score -2.572718
[LightGBM] [Info] Start training from score -2.544725
[LightGBM] [Info] Start training from score -2.544161
[LightGBM] [Info] Start training from score -2.547263
[LightGBM] [Info] Start training from score -2.552927
[LightGBM] [Info] Start training from score -2.595573
[LightGBM] [Info] Start training from score -2.382079
[LightGBM] [Info] Start training from score -2.553779

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/bd0cddec9e424fa4bd23561cf03d600f

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/bd0cddec9e424fa4bd23561cf03d600f
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.25368837711406983
COMET INFO:     precision     : 0.26482242684700047
COMET INFO:  

<Figure size 640x480 with 0 Axes>